In [6]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [7]:
import PrjtCF_module as cf

# Setting

In [8]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 100
print(pd.get_option("display.max_columns"), pd.get_option("display.max_rows"))

40 100


# Outline
* 주거시설 등 개발과 동시에 분양을 하는 사업의 재무모델
* PF대출을 통해 토지비 및 공사비 등 개발사업 자금을 조달
* 금융비용을 PF대출금으로 충당
* 분양대금의 일부는 사업비로, 일부는 대출금 상환재원으로 사용
* PF대출금은 필요에 따라 한도대로 인출

# Input Index Data

In [9]:
# 기간 Index 설정
# prjt index : 사업기간 전 기간에 대한 index로 모델의 base가 되는 index
# cstrn index : 공사기간에 대한 index로 착공일로부터 준공일까지의 기간
# loan index : 대출기간에 대한 index로 대출 실행일로부터 대출 상환일 까지의 기간
# sales index : 분양기간에 대한 index로 분양개시일로부터 분양종료일 까지의 기간
idx = cf.PrjtIndex(idxname=['prjt', 'cstrn', 'loan', 'sales'],
                   start=['2021-08', '2021-10', '2021-10', '2021-12'],
                   periods=[24+1, 18+1, 20+1, 16+1],
                   freq='M')

# 공정률 Series 설정 : cstrn index 기간 중 적용되는 기간별 공정률 설정
idx.prcs = Series(np.ones(len(idx.cstrn)) / len(idx.cstrn),
                  index=idx.cstrn.index)

# Input Financing Condition Data

In [10]:
equity = cf.Loan(idx, idx, amt_ntnl=10_000)

dct_loan = {}
tra = cf.Loan(idx, idx.loan, amt_ntnl=100_000,
              rate_fee = 0.01,
              rate_IR = 0.05)
dct_loan['tra'] = tra

trb = cf.Loan(idx, idx.loan, amt_ntnl=30_000,
              rate_fee = 0.03,
              rate_IR = 0.07)
dct_loan['trb'] = trb

loan = cf.Merge_loan(dct_loan)
loan.tra = loan.dct['tra']
loan.trb = loan.dct['trb']

# Input Sales Data

In [11]:
dct_sales = {}

# 분양매출 및 납입 일정
# 분양상품 A #
salesA = cf.Account(idx)
salesA.amt = 150_000 # Total sales amount
salesA.csh_idx = idx.sales[[0, 3, 6, 9, 12, 16]] # cashin schedule
salesA.csh_rate = Series([0.1, 0.1, 0.1, 0.1, 0.1, 0.5],
                         index=salesA.csh_idx) # cashin rate

# 분양상품 B #
salesB = cf.Account(idx)
salesB.amt = 70_000 # Total sales amount
salesB.csh_idx = idx.sales[[1,5,7,10,13,16]] # cashin schedule
salesB.csh_rate = Series([0.1, 0.1, 0.1, 0.1, 0.1, 0.5],
                         index=salesB.csh_idx) # cashin rate

# 분양매출 가정
salesA.sls_rate = Series(np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
                       index=idx.sales[[0, 2, 4, 6, 7, 8, 9, 12, 14, 16]])
salesA.sls_plan = salesA.amt * salesA.sls_rate

salesB.sls_rate = Series(np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
                       index=idx.sales[[1, 3, 5, 7, 10, 11, 12, 13, 15, 16]])
salesB.sls_plan = salesB.amt * salesB.sls_rate

# Make dictionary
dct_sales['salesA'] = salesA
dct_sales['salesB'] = salesB
sales = cf.Merge(dct_sales)
sales.salesA = sales.dct['salesA']
sales.salesB = sales.dct['salesB']

# Input Cost Data

In [12]:
dct_cost = {}

# 토지비: 최초 1회 지급
lnd = cf.Account(idx)
lnd.addscdd(idx.cstrn[0], 30_000)
dct_cost['lnd'] = lnd

# 공사비: 공정률에 따라 지급
cstrn = cf.Account(idx)
cstrn.addscdd(idx.cstrn.index, 50_000 * idx.prcs)
dct_cost['cstrn'] = cstrn

# 분양수수료: 분양계약 발생에 따라 지급
slsfee = cf.Account(idx)
slsfee.feerateA = 0.03
slsfee.feerateB = 0.05
slsfee.addscdd(salesA.sls_plan.index, salesA.sls_plan * slsfee.feerateA)
slsfee.addscdd(salesB.sls_plan.index, salesB.sls_plan * slsfee.feerateB)
dct_cost['slsfee'] = slsfee

# Make dictionary
cost = cf.Merge(dct_cost)
cost.lnd = cost.dct['lnd']
cost.cstrn = cost.dct['cstrn']
cost.slsfee = cost.dct['slsfee']

# Execution Cash Flow
### 1) 사전 설정

In [13]:
# Make accounts
dct_acc = {}
dct_acc['oprtg'] = cf.Account(idx)
dct_acc['sales'] = cf.Account(idx)
dct_acc['repay'] = cf.Account(idx)

acc = cf.Merge(dct_acc)
acc.oprtg = acc.dct['oprtg']
acc.sales = acc.dct['sales']
acc.repay = acc.dct['repay']

### 2) Cash Flow 실행

In [14]:
# Excute cash flow
for idxno in idx.index:
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno)
    loan.tra.set_wtdrbl_intldate(idxno)
    loan.trb.set_wtdrbl_intldate(idxno)
    
    #### Sales ####
    # 분양수입대금 sales schedul에 따라 운영계좌로 이체
    slsmngA = cf.sls_mngmnt(idxno, sales.salesA)
    slsmngA.make_sls_plan()
    slsmngA.rcv_slsamt(acc.sales)
    
    slsmngB = cf.sls_mngmnt(idxno, sales.salesB)
    slsmngB.make_sls_plan()
    slsmngB.rcv_slsamt(acc.sales)
    
    # 분양수입금 분배
    intl_bal_end = acc.sales.bal_end[idxno]
    amt_to_oprtg = intl_bal_end * 0.2
    amt_to_repay = intl_bal_end * 0.8
    acc.sales.send(idxno, amt_to_oprtg, acc.oprtg)
    acc.sales.send(idxno, amt_to_repay, acc.repay)
    
    # cost instance 생성, 조달금액 계산 후 대출인출액을 운영계좌 입금
    cstmng = cf.cst_mngmnt(idxno, cost, loan)
    wtdrw = cf.wtdrw_mngmnt(idxno, cstmng, acc.oprtg)
    wtdrw.wtdrw_equity(equity)
    wtdrw.wtdrw_loan(loan.trb)
    wtdrw.wtdrw_loan(loan.tra)
    
    # 운영계좌에서 토지비, 공사비 등 각종 비용 지출
    cstmng.pay_oprtcst(acc.oprtg)
    
    # 운영계좌에서 각종 금융비용 지출
    cstmng.pay_fnclcst(acc.oprtg)
    
    # 상환계좌 잔액으로 대출금 상환
    rpymngA = cf.repay_mngmnt(idxno, loan.tra)
    rpymngA.trsf_rpy(acc.oprtg, acc.repay)
    rpymngA.rpy_ntnl(acc.repay)
    
    rpymngB = cf.repay_mngmnt(idxno, loan.trb)
    rpymngB.trsf_rpy(acc.oprtg, acc.repay)
    rpymngB.rpy_ntnl(acc.repay)
    
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    loan.tra.setback_wtdrbl_mtrt(idxno)
    loan.trb.setback_wtdrbl_mtrt(idxno)

In [16]:
# 현금흐름 요약
rslt_df = DataFrame({('salesA', 'amt_add'):sales.salesA.amt_add[:],
                     ('salesA', 'amt_sub'):sales.salesA.amt_sub[:],
                     ('salesB', 'amt_add'):sales.salesB.amt_add[:],
                     ('salesB', 'amt_sub'):sales.salesB.amt_sub[:],
                     ('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_sub'):loan.tra.ntnl.amt_sub[:],
                     ('trb.ntnl', 'amt_sub'):loan.trb.ntnl.amt_sub[:],
                     ('oprtg', 'amt_add'):acc.oprtg.amt_add[:],
                     ('repay', 'amt_add'):acc.repay.amt_add[:],
                     ('tra.fee', 'amt_add'):loan.tra.fee.amt_add[:],
                     ('tra.IR', 'amt_add'):loan.tra.IR.amt_add[:],
                     ('trb.fee', 'amt_add'):loan.trb.fee.amt_add[:],
                     ('trb.IR', 'amt_add'):loan.trb.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('cost_slsfee', 'amt_add'):cost['slsfee'].amt_add[:],
                     ('oprtg', 'amt_sub'):acc.oprtg.amt_sub[:],
                     ('oprtg', 'bal_end'):acc.oprtg.bal_end[:],
                     ('repay', 'amt_sub'):acc.repay.amt_sub[:],
                     ('repay', 'bal_end'):acc.repay.bal_end[:],
                     ('tra.ntnl', 'amt_add'):loan.tra.ntnl.amt_add[:],
                     ('trb.ntnl', 'amt_add'):loan.trb.ntnl.amt_add[:],
                     })
rslt_df.fillna(0).applymap(lambda x: f"{x:,.0f}")

equity.ntnl  salesA          salesB         tra.ntnl trb.ntnl  \
               amt_sub amt_add amt_sub amt_add amt_sub  amt_sub  amt_sub   
2021-08-31      10,000       0       0       0       0        0        0   
2021-09-30           0       0       0       0       0        0        0   
2021-10-31           0       0       0       0       0        0   24,532   
2021-11-30           0       0       0       0       0        0    4,349   
2021-12-31           0  15,000   1,500       0       0    3,684    1,120   
2022-01-31           0       0       0   7,000     700    5,066        0   
2022-02-28           0  15,000   1,500       0       0    5,231        0   
2022-03-31           0       0   3,000   7,000     700    4,893        0   
2022-04-30           0  15,000   3,000       0       0    5,229        0   
2022-05-31           0       0       0   7,000   2,800    5,311        0   
2022-06-30           0  15,000   9,000       0       0    4,324        0   
2022-07-31           0  15,000   4,500   7,000   4,200    4,590        0   
2022-08-31           0  15,000   4,500       0       0    4,962        0   
2022-09-30           0  15,000  15,000       0       0    2,930        0   
2022-10-31           0       0       0   7,000   5,600    4,257        0   
2022-11-30           0       0       0   7,000   2,800    4,805        0   
2022-12-31           0  15,000  18,000   7,000   2,800    1,784        0   
2023-01-31           0       0       0   7,000   8,400    2,939        0   
2023-02-28           0  15,000   7,500       0       0    2,954        0   
2023-03-31           0       0       0   7,000   3,500    3,441        0   
2023-04-30           0  15,000  82,500   7,000  38,500        0        0   
2023-05-31           0       0       0       0       0        0        0   
2023-06-30           0       0       0       0       0        0        0   
2023-07-31           0       0       0       0       0        0        0   
2023-08-31           0       0       0       0       0        0        0   

             oprtg   repay tra.fee  tra.IR trb.fee  trb.IR cost_lnd  \
           amt_add amt_add amt_add amt_add amt_add amt_add  amt_add   
2021-08-31  10,000       0       0       0       0       0        0   
2021-09-30       0       0       0       0       0       0        0   
2021-10-31  24,532       0   1,000       0     900       0   30,000   
2021-11-30   4,349       0       0       0       0   1,717        0   
2021-12-31   5,103   1,200       0       0       0   2,022        0   
2022-01-31   5,206     560       0     124       0   2,100        0   
2022-02-28   5,531   1,200       0     349       0   2,100        0   
2022-03-31   5,633   2,960       0     551       0   2,100        0   
2022-04-30   5,829   2,400       0     648       0   2,100        0   
2022-05-31   5,871   2,240       0     789       0   2,100        0   
2022-06-30   6,124   7,200       0     943       0   2,100        0   
2022-07-31   6,330   6,960       0     799       0   2,100        0   
2022-08-31   5,862   3,600       0     680       0   2,100        0   
2022-09-30   5,930  12,000       0     748       0   2,100        0   
2022-10-31   5,377   4,480       0     295       0   2,100        0   
2022-11-30   5,365   2,240       0     284       0   2,100        0   
2022-12-31   5,944  16,640       0     412       0   2,100        0   
2023-01-31   4,619   6,720       0       0       0   1,637        0   
2023-02-28   4,454   6,000       0       0       0   1,372        0   
2023-03-31   4,141   2,800       0       0       0   1,159        0   
2023-04-30  24,200  96,800       0      32       0   1,159        0   
2023-05-31       0       0       0       0       0       0        0   
2023-06-30       0       0       0       0       0       0        0   
2023-07-31       0       0       0       0       0       0        0   
2023-08-31       0       0       0       0       0       0        0   

           cost_cstrn cost_slsfee   oprt